# Environment setting / 環境設定

In [ ]:
import os  # noqa: I001
import sys
from pathlib import Path

# Handle utils.py for Colab
if "COLAB_GPU" in os.environ:
    import urllib.request

    demo_utils_url = (
        "https://raw.githubusercontent.com/nics-tw/petsard/main/demo/demo_utils.py"
    )
    # Download and save to file instead of exec
    demo_utils_content = urllib.request.urlopen(demo_utils_url).read().decode("utf-8")
    with open("demo_utils.py", "w") as f:
        f.write(demo_utils_content)
    print("✅ demo_utils.py downloaded")

else:
    # demo_utils.py search for local
    for p in [Path.cwd()] + list(Path.cwd().parents)[:10]:
        utils_path = p / "demo_utils.py"
        if utils_path.exists() and "demo" in str(utils_path):
            sys.path.insert(0, str(p))
            break

## Quick setup / 快速設定: Preprocessor YAML - Scaler

In [2]:
from demo_utils import display_results, display_yaml_info, quick_setup  # noqa: I001

is_colab, branch, yaml_path, Executor = quick_setup(
    config_file=[
        "preprocessor_scaler-field-specific.yaml",
        "preprocessor_scaler-time-anchor-scaling.yaml",
    ],
    benchmark_data=None,
    petsard_branch="main",
)

✅ Changed working directory to demo: petsard/demo
   📁 Notebook location: demo/petsard-yaml/preprocessor-yaml/
   🔍 YAML search priority: 
      1. demo/petsard-yaml/preprocessor-yaml/
      2. demo/
   💾 Output files will be saved in: demo/
🚀 PETsARD v1.9.0
📅 2025-11-19 11:22:41 UTC+8
🔧 Added to Python path: petsard/demo/petsard-yaml/preprocessor-yaml
📁 Processing configuration files from subfolder: petsard-yaml/preprocessor-yaml
✅ Found configuration (1/2): petsard/demo/petsard-yaml/preprocessor-yaml/preprocessor_scaler-field-specific.yaml
✅ Found configuration (2/2): petsard/demo/petsard-yaml/preprocessor-yaml/preprocessor_scaler-time-anchor-scaling.yaml


# Execution and Result / 執行與結果

## Customizing Scaling for Specific Fields / 自訂特定欄位的縮放

In [3]:
display_yaml_info(yaml_path[0])
exec_now = Executor(yaml_path[0])
exec_now.run()
display_results(exec_now.get_result())

📋 YAML Configuration Files / YAML 設定檔案

📄 File: preprocessor_scaler-field-specific.yaml
📁 Path: petsard/demo/petsard-yaml/preprocessor-yaml/preprocessor_scaler-field-specific.yaml

⚙️ Configuration content / 設定內容:
----------------------------------------
---
Loader:
  load_benchmark_with_schema:
    filepath: benchmark://adult-income
    schema: benchmark://adult-income_schema

Preprocessor:
  scaling-specific:
    sequence:
      - scaler
    config:
      scaler:
        age: 'scaler_minmax'          # Min-Max scaling
        fnlwgt: 'scaler_standard'     # Standardization
        educational-num: 'scaler_log' # Log transformation
        capital-loss: None            # No scaling for categorical field

Reporter:
  save_data:
    method: save_data
    source:
      - Preprocessor
  save_schema:
    method: save_schema
    source:
      - Loader
      - Preprocessor
...
📊 Execution Results / 執行結果

[1] Loader[load_benchmark_with_schema]_Preprocessor[scaling-specific]_Reporter[save_data

## Time Anchor Scaling / 時間錨點縮放

In [4]:
display_yaml_info(yaml_path[1])
exec_now = Executor(yaml_path[1])
exec_now.run()
display_results(exec_now.get_result())

📋 YAML Configuration Files / YAML 設定檔案

📄 File: preprocessor_scaler-time-anchor-scaling.yaml
📁 Path: petsard/demo/petsard-yaml/preprocessor-yaml/preprocessor_scaler-time-anchor-scaling.yaml

⚙️ Configuration content / 設定內容:
----------------------------------------
---
Loader:
  load_benchmark_with_schema:
    filepath: benchmark://best-practices_multi-table

Preprocessor:
  time_scaling:
    sequence:
      - scaler
    config:
      scaler:
        established_date:
          method: 'scaler_timeanchor'
          reference:                   # Reference time field
            - 'first_apply_apply_date'
            - 'first_apply_approval_date'
            - 'latest_apply_apply_date'
            - 'latest_apply_approval_date'
          unit: 'D'                    # Unit: days

Reporter:
  save_data:
    method: save_data
    source:
      - Preprocessor
  save_schema:
    method: save_schema
    source:
      - Loader
      - Preprocessor
...
📊 Execution Results / 執行結果

[1] Loader[loa